# Network comparison with more data - LSTM vs GRU 

In [0]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

In [0]:
""" optional: select GPU Hardware acceleration """
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [3]:
!pip install quandl

  Created wheel for inflection: filename=inflection-0.3.1-cp36-none-any.whl size=6077 sha256=3306bf1cd5794a7bd57798e4bb0e11bf800089ed2025a547c8a84626a06d423f
  Stored in directory: /root/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [0]:
""" Import required classes """
import quandl 
from dataprep_class import Dataprep

In [18]:
quandl.ApiConfig.api_key = "Ha9a4yRKHowJC_sQxSPD"

""" commodity prices """
oil = quandl.get("EIA/PET_RWTC_D")
print(oil.head())
print(oil.shape)

gold = quandl.get("LBMA/GOLD", start_date = "1960-12-01")
print(gold.head())
print(gold.shape)

""" equity prices """
msft = quandl.get("EOD/MSFT")
print(msft.head())
print(msft.shape)

hd = quandl.get("EOD/HD")
print(hd.head())
print(hd.shape)

dis = quandl.get("EOD/DIS")
print(dis.head())
print(dis.shape)

ba = quandl.get("EOD/BA")
print(ba.head())
print(ba.shape)

aapl = quandl.get("EOD/aapl")
print(aapl.head())
print(aapl.shape)

            Value
Date             
1986-01-02  25.56
1986-01-03  26.00
1986-01-06  26.53
1986-01-07  25.85
1986-01-08  25.87
(8652, 1)
            USD (AM)  USD (PM)  GBP (AM)  GBP (PM)  EURO (AM)  EURO (PM)
Date                                                                    
1968-01-02     35.18       NaN    14.641       NaN        NaN        NaN
1968-01-03     35.16       NaN    14.617       NaN        NaN        NaN
1968-01-04     35.14       NaN    14.603       NaN        NaN        NaN
1968-01-05     35.14       NaN    14.597       NaN        NaN        NaN
1968-01-08     35.14       NaN    14.586       NaN        NaN        NaN
(13210, 6)
               Open   High    Low  ...    Adj_Low  Adj_Close   Adj_Volume
Date                               ...                                   
2013-09-03  31.7500  32.07  31.29  ...  27.160215  27.672344  154507000.0
2013-09-04  31.3900  31.47  31.11  ...  27.003972  27.077754  142320600.0
2013-09-05  31.1000  31.44  30.95  ...  26.865

In [24]:
print(ba.shape[1])

12


In [25]:
data = ba
colidx = 1
data = data.iloc[::-1]
data = data.reset_index()
data.drop('Date', axis = 1, inplace = True)
colidx -= 1
data = data.to_numpy()
data = data[:, colidx]

print(data[0:10,...])

[295.81  296.41  295.54  295.135 298.04  298.88  296.35  295.08  296.05
 292.64 ]
